Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Authentication in Azure Machine Learning

This notebook shows you how to authenticate to your Azure ML Workspace using

 1. Interactive Login Authentication
 2. Azure CLI Authentication
 3. Service Principal Authentication
 
The interactive authentication is suitable for local experimentation on your own computer. Azure CLI authentication is suitable if you are already using Azure CLI for managing Azure resources, and want to sign in only once. The Service Principal authentication is suitable for automated workflows, for example as part of Azure Devops build.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azureml.png)

In [ ]:
from azureml.core import Workspace

### Interactive Authentication

Interactive authentication is the default mode when using Azure ML SDK.

When you connect to your workspace using workspace.from_config, you will get an interactive login dialog.

In [ ]:
ws = Workspace.from_config()

Also, if you explicitly specify the subscription ID, resource group and resource group, you will get the dialog.

In [ ]:
ws = Workspace(subscription_id="my-subscription-id",
               resource_group="my-ml-rg",
               workspace_name="my-ml-workspace")

Note the user you're authenticated as must have access to the subscription and resource group. If you receive an error

```
AuthenticationException: You don't have access to xxxxxx-xxxx-xxx-xxx-xxxxxxxxxx subscription. All the subscriptions that you have access to = ...
```

check that the you used correct login and entered the correct subscription ID.

In some cases, you may see a version of the error message containing text: ```All the subscriptions that you have access to = []```

In such a case, you may have to specify the tenant ID of the Azure Active Directory you're using. An example would be accessing a subscription as a guest to a tenant that is not your default. You specify the tenant by explicitly instantiating _InteractiveLoginAuthentication_ with tenant ID as argument ([see instructions how to obtain tenant Id](#get-tenant-id)).

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id="my-tenant-id")

ws = Workspace(subscription_id="my-subscription-id",
               resource_group="my-ml-rg",
               workspace_name="my-ml-workspace",
               auth=interactive_auth)

### Azure CLI Authentication

If you have installed azure-cli package, and used ```az login``` command to log in to your Azure Subscription, you can use _AzureCliAuthentication_ class.

Note that interactive authentication described above won't use existing Azure CLI auth tokens. 

In [ ]:
from azureml.core.authentication import AzureCliAuthentication

cli_auth = AzureCliAuthentication()

ws = Workspace(subscription_id="my-subscription-id",
               resource_group="my-ml-rg",
               workspace_name="my-ml-workspace",
               auth=cli_auth)

print("Found workspace {} at location {}".format(ws.name, ws.location))

### Service Principal Authentication

When setting up a machine learning workflow as an automated process, we recommend using Service Principal Authentication. This approach decouples the authentication from any specific user login, and allows managed access control.

Note that you must have administrator privileges over the Azure subscription to complete these steps.

The first step is to create a service principal. First, go to [Azure Portal](https://portal.azure.com), select **Azure Active Directory** and **App Registrations**. Then select **+New application registration**, give your service principal a name, for example _my-svc-principal_. You can leave application type as is, and specify a dummy value for Sign-on URL, such as _https://invalid_.

Then click **Create**.

![service principal creation]<img src="images/svc-pr-1.PNG">

The next step is to obtain the _Application ID_ (also called username) and create _password_ for the service principal.

From the page for your newly created service principal, copy the _Application ID_. Then select **Settings** and **Keys**, write a description for your key, and select duration. Then click **Save**, and copy the _password_ to a secure location.

![application id and password](images/svc-pr-2.PNG)

<a id ="get-tenant-id"></a>

Also, you need to obtain the tenant ID of your Azure subscription. Go back to **Azure Active Directory**, select **Properties** and copy _Directory ID_.

![tenant id](images/svc-pr-3.PNG)

Finally, you need to give the service principal permissions to access your workspace. Navigate to **Resource Groups**, to the resource group for your Machine Learning Workspace. 

Then select **Access Control (IAM)** and **Add a role assignment**. For _Role_, specify which level of access you need to grant, for example _Contributor_. Start entering your service principal name and once it is found, select it, and click **Save**.

![add role](images/svc-pr-4.PNG)

Now you are ready to use the service principal authentication. For example, to connect to your Workspace, see code below and enter your own values for tenant ID, application ID, subscription ID, resource group and workspace.

**We strongly recommended that you do not insert the secret password to code**. Instead, you can use environment variables to pass it to your code, for example through Azure Key Vault, or through secret build variables in Azure DevOps. For local testing, you can for example use following PowerShell command to set the environment variable.

```
$env:AZUREML_PASSWORD = "my-password"
```

In [ ]:
import os
from azureml.core.authentication import ServicePrincipalAuthentication

svc_pr_password = os.environ.get("AZUREML_PASSWORD")

svc_pr = ServicePrincipalAuthentication(
    tenant_id="my-tenant-id",
    service_principal_id="my-application-id",
    service_principal_password=svc_pr_password)


ws = Workspace(
    subscription_id="my-subscription-id",
    resource_group="my-ml-rg",
    workspace_name="my-ml-workspace",
    auth=svc_pr
    )

print("Found workspace {} at location {}".format(ws.name, ws.location))